## ATIVIDADE 1 - Web Scraping em Ambiente Controlado

Importações de **bibliotecas** utilizadas

In [11]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import time
import csv

Definindo as **variáveis globais** que serão utilizadas ao longo do código

In [12]:
baseUrl = 'http://127.0.0.1:8000/places/default'

ids = ['places_country__row', 'places_currency_name__row', 'places_continent__row', 'places_neighbours__row']
pages = set()
info = []

## Funções

Função para pegar os links de páginas dos **países** (a partir da página *index*)

In [13]:
def getLinks(pageUrl):
    global pages, baseUrl
    fullUrl = urljoin(baseUrl, pageUrl)

    try:
        html = urlopen(fullUrl)
        bs = BeautifulSoup(html, 'html.parser')

        for link in bs.find_all('a'):
            href = link.get('href')
            newPage = urljoin(baseUrl, href)
            if newPage not in pages and 'user' not in href:
                if 'index' in href or 'view' in href:
                    pages.add(newPage)
                    getLinks(href)
    except HTTPError as e:
        print(f'Erro HTTP [{e.code}] tentando acessar [{fullUrl}]')
    except Exception as e:
        print(f'Deu pau :( [{e}]')

Função **pega_nome** trabalha juntamente com a **procura_info**, onde a última pega todas as informações necessárias das páginas - *Nome, Continente, Moeda* - e a primeira, além de entrar nos links dos países vizinho, pega o nome deles

In [14]:
def pega_nome(classe):
    aux = []
    for vizinho in classe.find_all('a'):
        url = urljoin(baseUrl, vizinho['href'])
        html = urlopen(url)
        bs = BeautifulSoup(html, 'html.parser')
        
        if 'iso' in url:
            linha = bs.find(id='places_country__row')
            if linha:
                nome = linha.find(class_='w2p_fw')
                if nome:
                    aux.append(nome.text.strip())
            
    return aux        

In [15]:
def procura_info(aux, bs):
    global ids, info
    
    for i in ids:
        linha = bs.find(id=i)
        classe = linha.find(class_='w2p_fw')

        if i == 'places_neighbours__row':
            aux.append(pega_nome(classe))
        else:
            aux.append(classe.text.strip())

    aux.append(time.ctime())
    info.append(aux)

Salva as informações presentes no array em um **.csv**

In [16]:
def salva_csv():
    global info
    colunas = ['País', 'Moeda', 'Continente', 'Países Vizinhos', 'Horário']

    with open('Paises.csv', 'w', newline='') as arq:
        w = csv.writer(arq)
        w.writerow(colunas)
        w.writerows(info)

print('Deu certo')

Deu certo


## Main

Partes de código em que as funções são chamadas e testes são realizados

In [17]:
getLinks('/places/default/index')

In [18]:
for page in pages:
    if 'index' not in page:
        aux = []
        html = urlopen(page)
        bs = BeautifulSoup(html, 'html.parser')

        procura_info(aux, bs)
    

In [19]:
for i in info:
    print(i)

['British Indian Ocean Territory', 'Dollar', 'AS', [], 'Mon Aug 26 18:23:41 2024']
['Cambodia', 'Riels', 'AS', ['Laos', 'Thailand', 'Vietnam'], 'Mon Aug 26 18:23:42 2024']
['Macedonia', 'Denar', 'EU', ['Albania', 'Greece', 'Serbia and Montenegro', 'Bulgaria', 'Serbia', 'Kosovo'], 'Mon Aug 26 18:23:44 2024']
['Central African Republic', 'Franc', 'AF', ['Chad', 'Sudan', 'Democratic Republic of the Congo', 'South Sudan', 'Cameroon', 'Republic of the Congo'], 'Mon Aug 26 18:23:46 2024']
['Guinea', 'Franc', 'AF', ['Liberia', 'Senegal', 'Sierra Leone', 'Ivory Coast', 'Guinea-Bissau', 'Mali'], 'Mon Aug 26 18:23:48 2024']
['Spain', 'Euro', 'EU', ['Andorra', 'Portugal', 'Gibraltar', 'France', 'Morocco'], 'Mon Aug 26 18:23:50 2024']
['Armenia', 'Dram', 'AS', ['Georgia', 'Iran', 'Azerbaijan', 'Turkey'], 'Mon Aug 26 18:23:51 2024']
['Falkland Islands', 'Pound', 'SA', [], 'Mon Aug 26 18:23:52 2024']
['Mexico', 'Peso', 'NA', ['Guatemala', 'United States', 'Belize'], 'Mon Aug 26 18:23:53 2024']
['Vat

In [20]:
salva_csv()